In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import joblib
import random
import warnings
import os

# Suppress warnings for a cleaner output
warnings.filterwarnings("ignore", category=UserWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

# --- Configuration (MUST match your training setup) ---
MODEL_FILE = '/content/smartwatch_model_v2.h5'
SCALER_FILE = '/content/smartwatch_scaler.joblib'
SAMPLING_INTERVAL_HOURS = 2
TIMESTEPS_PER_DAY = 24 // SAMPLING_INTERVAL_HOURS
SEQUENCE_DAYS = 14
SEQUENCE_TIMESTEPS = SEQUENCE_DAYS * TIMESTEPS_PER_DAY
N_FEATURES = 8 # Number of features in the smartwatch dataset

def generate_sample_sequence(failure_type="healthy"):
    """
    Generates a sample sequence of raw smartwatch data for different failure scenarios.
    """
    print(f"\nGenerating a sample sequence for scenario: '{failure_type}'...")
    raw_data = []

    for i in range(SEQUENCE_TIMESTEPS):
        # Generate baseline healthy values
        healthy_reading = {
            'battery_level': random.uniform(30, 90),
            'heart_rate_bpm': random.uniform(60, 110),
            'steps_per_hour': random.randint(0, 500),
            'gps_active': 0,
            'screen_on_time_minutes': random.uniform(1, 10),
            'ambient_temp_c': 25.0,
            'water_pressure_atm': 1.0 + random.uniform(-0.01, 0.01),
            'fall_detection_events': 0
        }

        # Apply specific failure logic
        if failure_type == "battery_failure":
            # Battery drains unusually fast
            healthy_reading['battery_level'] *= (1 - (i / SEQUENCE_TIMESTEPS) * 0.7) # Degrades to 30% of normal

        elif failure_type == "heart_rate_sensor_failure":
            # Heart rate sensor becomes erratic and reads high
            healthy_reading['heart_rate_bpm'] += 50 + random.uniform(-20, 20) # Erratic and high readings

        elif failure_type == "water_seal_failure":
            # Water pressure sensor gives erratic readings, indicating a seal breach
            healthy_reading['water_pressure_atm'] += random.uniform(-0.5, 0.5)

        raw_data.append(healthy_reading)

    return raw_data

def run_diagnostic_analysis(model, scaler, raw_data_sequence):
    """
    Generates a diagnostic report for smartwatches using feature perturbation analysis.
    """
    feature_columns = [
        'battery_level', 'heart_rate_bpm', 'steps_per_hour', 'gps_active',
        'screen_on_time_minutes', 'ambient_temp_c', 'water_pressure_atm',
        'fall_detection_events'
    ]

    df = pd.DataFrame(raw_data_sequence)
    scaled_features = scaler.transform(df[feature_columns])
    original_sequence = np.array([scaled_features])
    base_risk_score = model.predict(original_sequence, verbose=0)[0][0]

    if base_risk_score < 0.5:
        print("--- Diagnostic Report ---")
        print("Prediction: Normal Operation")
        print(f"Risk Score: {base_risk_score:.4f}\nNo anomalies detected.")
        return

    print(f"High risk detected ({base_risk_score:.4f}). Running perturbation analysis...")

    feature_impacts = {}
    # Define "healthy" values for each feature (approximates the mean of scaled healthy data)
    healthy_values_scaled = {
        'battery_level': 0.8, 'heart_rate_bpm': 0.4, 'steps_per_hour': 0.2, 'gps_active': 0.0,
        'screen_on_time_minutes': 0.3, 'ambient_temp_c': 0.5, 'water_pressure_atm': 0.5,
        'fall_detection_events': 0.0
    }

    for i, feature_name in enumerate(feature_columns):
        perturbed_sequence = original_sequence.copy()
        perturbed_sequence[:, :, i] = healthy_values_scaled[feature_name]
        new_risk_score = model.predict(perturbed_sequence, verbose=0)[0][0]
        impact = base_risk_score - new_risk_score
        feature_impacts[feature_name] = impact

    impact_series = pd.Series(feature_impacts).sort_values(ascending=False)

    cause_map = {
        "battery_level": "Severe Battery Degradation",
        "heart_rate_bpm": "Erratic Heart Rate Sensor",
        "water_pressure_atm": "Potential Water Seal Failure",
        "fall_detection_events": "Anomalous Fall Detection Events",
        "steps_per_hour": "Anomalous Activity Tracking",
        "gps_active": "GPS System Malfunction",
        "screen_on_time_minutes": "Excessive Screen On Time",
        "ambient_temp_c": "Overheating"
    }

    primary_cause_feature = impact_series.index[0]
    secondary_cause_feature = impact_series.index[1]

    print("\n" + "="*40)
    print("  Smartwatch Predictive Diagnostic Report")
    print("="*40)
    print(f"Prediction: Failure Imminent")
    print(f"Risk Score: {base_risk_score:.4f}")
    print("\n--- Probable Causes (based on Perturbation Analysis) ---")
    print(f"1. Primary Cause: {cause_map.get(primary_cause_feature, 'Unknown Factor')}")
    print(f"   - Details: Neutralizing the '{primary_cause_feature}' values caused the largest drop in the risk score.")
    print(f"2. Secondary Cause: {cause_map.get(secondary_cause_feature, 'Unknown Factor')}")
    print(f"   - Details: Neutralizing the '{secondary_cause_feature}' values caused the second-largest drop in risk.")
    print("\n--- Recommendation ---")
    print("Advise user to sync their data, charge the device, and contact support if issues persist.")
    print("="*40)


if __name__ == "__main__":
    try:
        model = keras.models.load_model(MODEL_FILE)
        scaler = joblib.load(SCALER_FILE)
    except Exception as e:
        print(f"Fatal Error: Could not load model or scaler. {e}")
        exit()

    # --- Test different failure scenarios ---

    # Scenario 1: A battery failure
    battery_fail_sequence = generate_sample_sequence(failure_type="battery_failure")
    run_diagnostic_analysis(model, scaler, battery_fail_sequence)

    # Scenario 2: An erratic heart rate sensor
    hr_sensor_fail_sequence = generate_sample_sequence(failure_type="heart_rate_sensor_failure")
    run_diagnostic_analysis(model, scaler, hr_sensor_fail_sequence)

    # Scenario 3: A water seal failure
    seal_fail_sequence = generate_sample_sequence(failure_type="water_seal_failure")
    run_diagnostic_analysis(model, scaler, seal_fail_sequence)


Generating a sample sequence for scenario: 'battery_failure'...
High risk detected (0.9593). Running perturbation analysis...

  Smartwatch Predictive Diagnostic Report
Prediction: Failure Imminent
Risk Score: 0.9593

--- Probable Causes (based on Perturbation Analysis) ---
1. Primary Cause: Severe Battery Degradation
   - Details: Neutralizing the 'battery_level' values caused the largest drop in the risk score.
2. Secondary Cause: Anomalous Activity Tracking
   - Details: Neutralizing the 'steps_per_hour' values caused the second-largest drop in risk.

--- Recommendation ---
Advise user to sync their data, charge the device, and contact support if issues persist.

Generating a sample sequence for scenario: 'heart_rate_sensor_failure'...
High risk detected (0.9641). Running perturbation analysis...

  Smartwatch Predictive Diagnostic Report
Prediction: Failure Imminent
Risk Score: 0.9641

--- Probable Causes (based on Perturbation Analysis) ---
1. Primary Cause: Erratic Heart Rate Se